In [ ]:
# default_exp utils.lessrandom

In [ ]:
#hide
from nbdev.showdoc import *

# Less Random
> A random state manager which provides some reproducibility without sacrificing potential training speed.

Unlike fast.ai's [`no_random`](https://docs.fast.ai/torch_core.html#no_random), `less_random` does not set `torch.backends.cudnn.benchmark = False` so it's possible to train faster. Training runs on the same GPU, PyTorch, & CUDA setup should be reproducable, but differnet hardware/software setup will probably have less reproducability then using `no_random`.

In [ ]:
#export
import torch, random
import numpy as np
from fastcore.foundation import contextmanager
from fastai.callback.core import set_random_states, get_random_states

In [ ]:
#export
@contextmanager
def less_random(seed=42, deterministic=True):
    "Stores and retrieves state of random number generators. Sets random seed for `random`, `torch`, and `numpy`. Does not set `torch.backends.cudnn.benchmark = False`"
    states = get_random_states()
    
    try: torch.manual_seed(seed)
    except NameError: pass
    try: torch.cuda.manual_seed_all(seed)
    except NameError: pass
    try: np.random.seed(seed%(2**32-1))
    except NameError: pass
    random.seed(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True
        
    try:
        yield #we are managing global variables
    finally:
        set_random_states(**states)